<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Сергей! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Вывод:</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Обучим-следующие-модели:" data-toc-modified-id="Обучим-следующие-модели:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучим следующие модели:</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import nltk
import pandas as pd
import re
import warnings


from IPython.display import display
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from tqdm import notebook

m = WordNetLemmatizer() 

warnings.filterwarnings('ignore')

notebook.tqdm.pandas()

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /Users/vs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/vs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vs/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

In [2]:
random_state = 12345

In [3]:
data = pd.read_csv('toxic_comments.csv')

In [4]:
display(data.head())
data.info()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> Метод info() не нуждается в display(), и возвращает None.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Данные загружены корректно, первичный осмотр проведен.</font>

In [5]:
display(data['toxic'].value_counts())
data['toxic'].value_counts(normalize=True).map('{:.2%}'.format)

toxic
0    143106
1     16186
Name: count, dtype: int64

toxic
0    89.84%
1    10.16%
Name: proportion, dtype: object

Целевой признак с очень большим дисбалансом.

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Мы обнаружили серьёзный дисбаланс при исследовании данных. Как дополнительные материалы, рекомендую статью <a href='https://dyakonov.org/2021/05/27/imbalance/'>Дисбаланс классов</a>, очень классная, как и весь блог Дьяконова. Ещё такой <a href='https://github.com/Dyakonov/ml_hacks/blob/master/book_disbalance_public_v1.ipynb'>ноутбук</a> есть.</font>

### Подготовка данных

In [6]:
def text_tokenize(text):
    
    tokenized = nltk.word_tokenize(text)
    tagged = nltk.pos_tag(tokenized)

    lemmatizer = WordNetLemmatizer()
    
    lemmatized_words = []
    for word, tag in tagged:
        pos = get_wordnet_pos(tag)
        lemmatized_word = lemmatizer.lemmatize(word, pos=pos) if pos else lemmatizer.lemmatize(word)
        lemmatized_words.append(lemmatized_word)

    result = ' '.join(lemmatized_words)
    result = re.sub(r'[^a-zA-Z]', ' ', result)
    result = ' '.join(result.split())

    return result

def get_wordnet_pos(tag):
    if tag.startswith('N'):
        return 'n'  # Noun
    elif tag.startswith('V'):
        return 'v'  # Verb
    elif tag.startswith('R'):
        return 'r'  # Adverb
    elif tag.startswith('J'):
        return 'a'  # Adjective
    else:
        return None


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Очистка сделана верно! Мы оставили только символы Латинского алфавита.</font> \
<font color='darkorange'>Можно ещё привести символы к одному регистру. Это уменьшит словарь уникальных слов.</font>

In [7]:
display(data['text'][0])

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Здорово что выводишь данные, так удобно отлаживать код, сразу видно, как работает функция.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'>~~Обрати внимание, не все слова приведены к начальным формам. Чтобы корректно обработались все части речи, для WordNetLemmatizer() нужно использовать POS-теги (Part of Speech, части речи). Примеры работы с WordNetLemmatizer(), а также с другими инструментами для лемматизации, можно найти в [этой статье](https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/)~~</font>

<font color='purple'><b>Комментарий студента</b></font>
Вроде как сделал)

In [8]:
try:
    df = pd.read_csv('data.csv')
    print('data is ready', data.shape)
except:
    notebook.tqdm.pandas() 
    data['text_final'] = data['text'].progress_apply(text_tokenize)
    try:
        data.to_csv('data.csv', index=False)
    except:
        print('Ошибка сохранения файла')

  0%|          | 0/159292 [00:00<?, ?it/s]

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ❌\
<font color='red'> Обрати внимание, в тетрадку загружаются модели 'wordnet', 'punkt', 'averaged_perceptron_tagger' при обработке каждой строки в датасете. Кроме увеличения времени обработки и вустую траты ресурсов сети и процессора, перегружается сама тетрадка, ведь всё что написано под ячейкой сохраняется вместе в файле .ipynb. В итоге платформа Практикума не справляется с таким файлом.\
Загрузка моделей выше должна быть венесена из функции. Это делается один раз. (можно в ячейке с импортами.)></font>

<font color='purple'><b>Комментарий студента 2</b></font>
Поправил

In [9]:
display(data['text_final'][0])

'Explanation Why the edits make under my username Hardcore Metallica Fan be revert They be n t vandalisms just closure on some GAs after I vote at New York Dolls FAC And please do n t remove the template from the talk page since I m retired now'

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Отлично! Теперь видно, что слова в начальных формах.</font>

In [10]:
features = data['text_final']
target = data['toxic']

In [11]:
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state=12345, 
                                                                            stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(features_test, target_test, random_state=12345, test_size=.5)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(119469,)
(119469,)
(19911,)
(19911,)
(19912,)
(19912,)


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

In [12]:
print('Распределение классов в обучающей выборке:')
print(target_train.value_counts()[0] / target_train.value_counts().sum())
print(target_train.value_counts()[1] / target_train.value_counts().sum())
print()
print('Распределение классов в валидационной выборке:')
print(target_valid.value_counts()[0] / target_valid.value_counts().sum())
print(target_valid.value_counts()[1] / target_valid.value_counts().sum())
print()
print('Распределение классов в тестовой выборке:')
print(target_test.value_counts()[0] / target_test.value_counts().sum())
print(target_test.value_counts()[1] / target_test.value_counts().sum())

Распределение классов в обучающей выборке:
0.8983836811222995
0.10161631887770049

Распределение классов в валидационной выборке:
0.8974436241273668
0.10255637587263322

Распределение классов в тестовой выборке:
0.8993571715548413
0.1006428284451587


<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~Метод .astype('U') кодирует текст в максимально широкий Юникод U32, содержащий все возможные символы и эмоджи. Такое форматирование здесь не имеет смысла, так как после очистки осталась только латинница (тексты, в основном, и так уже в Юникоде "utf-8"). Ни одна из используемых в проекте моделей не требует кодировки в Юникод. Можно обойтись и без него.~~</font>

<font color='purple'><b>Комментарий студента</b></font>
Убрал

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> 👍</font>

In [13]:
nltk.download('stopwords')
stopwords = list(set(nltk_stopwords.words('english')))
display(len(stopwords))

[nltk_data] Downloading package stopwords to /Users/vs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [14]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Убрали частые неинформативные слова!</font>

In [15]:
features_train_data_tf_idf = tf_idf.fit_transform(features_train)
features_valid_data_tf_idf = tf_idf.transform(features_valid)
features_test_data_tf_idf = tf_idf.transform(features_test)

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Классно, что векторизатор был обучен только на тренировочной части данных!</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> Можно объединить Векторизатор с моделью через Pipeline. Так можно избежать утечек даже при кроссвалидации моделей.<br> Материалы по Pipeline:<br> [О Пайплайн](https://dzen.ru/a/YBBkKJBsUV9MPret)<br>

[Примеры работы с текстами](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html)</font>

In [16]:
print('TF-IDF train')
print(features_train_data_tf_idf.shape)
print(features_train_data_tf_idf[:5].toarray())
print()
print('TF-IDF valid')
print(features_valid_data_tf_idf.shape)
print(features_valid_data_tf_idf[:5].toarray())
print()
print('TF-IDF test')
print(features_test_data_tf_idf.shape)
print(features_test_data_tf_idf[:5].toarray())

TF-IDF train
(119469, 136275)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

TF-IDF valid
(19911, 136275)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

TF-IDF test
(19912, 136275)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Вывод:

* Токенизировали и лемматизировали столбец text и добавили данные в столбец text_final
* Провели оценку важности слов определением величины TF-IDF
* Проверили данные после векторизации

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

## Обучение

## Обучим следующие модели:

* LogisticRegression
* RandomForestClassifier
* DecisionTreeClassifier

In [17]:
model = LogisticRegression(max_iter=500, random_state=random_state, class_weight='balanced', n_jobs=-1)
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_valid_data_tf_idf)
    
avg_f1_score = f1_score(target_valid, predictions)

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~Если модель оценивается методом predict, то нужна ещё одна независимая выборка для теста. Или можно использовать кроссвалидацию, тогда отдельная выборка генерится автоматически.~~</font>

<font color='purple'><b>Комментарий студента</b></font>
Поправил

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> 👍</font>

In [18]:
results = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['LogisticRegression'], 
                        'Test f1 score' : round(avg_f1_score, 2)})

In [19]:
model = RandomForestClassifier(random_state=random_state, class_weight = 'balanced', n_jobs=-1)
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_valid_data_tf_idf)
    
avg_f1_score = f1_score(target_valid, predictions)

In [20]:
results2 = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['RandomForestClassifier'], 
                        'Test f1 score' : round(avg_f1_score, 2)})

In [21]:
model = DecisionTreeClassifier(random_state=random_state, class_weight = 'balanced')
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_valid_data_tf_idf)
    
avg_f1_score = f1_score(target_valid, predictions)

In [22]:
results3 = pd.DataFrame({'Preprocessing model' : ['TF-IDF'],
                        'Learning model' : ['DecisionTreeClassifier'], 
                        'Test f1 score' : round(avg_f1_score, 2)})

In [23]:
results = pd.concat([results, results2], ignore_index=True)
results = pd.concat([results, results3], ignore_index=True)

display(results)

,Preprocessing model,Learning model,Test f1 score
0,TF-IDF,LogisticRegression,0.75
1,TF-IDF,RandomForestClassifier,0.61
2,TF-IDF,DecisionTreeClassifier,0.64


<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'> Отлично, что есть табличка результатов для сравнения.</font>

In [24]:
model = LogisticRegression(random_state=random_state, class_weight='balanced', n_jobs=-1)
model.fit(features_train_data_tf_idf, target_train)
predictions = model.predict(features_test_data_tf_idf)
    
round(f1_score(target_test, predictions), 2)

/Users/vs/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.75

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='darkorange'> Названия моделей одинаковые.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'> ~~Выбирать модели на тестовых данных нельзя - так мы подстраиваемся конкретно под тестовую выборку. Тестирование проводится только для лучшей модели отобранной по результатам кросс-валидации \ ввалидации.~~</font>

<font color='purple'><b>Комментарий студента</b></font>
Поправил

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

## Выводы

* Данные загружены, состоят из 159571 строк и 2 столбцов
* Столбец toxic является целевым признаком с выраженым дисбалансом классов
* Токенизировали и лемматизировали столбец text и добавим данные в столбец text_final
* Провели оценку важности слов определением величины TF-IDF
* Проверили данные после векторизации
* Обучили следующие модели:
    * LogisticRegression
    * RandomForestClassifier
    * DecisionTreeClassifier
* Лучший результат метрики F1-score 0.75 показала модель LogisticRegression
* Другие модели не прошли по условию F1-score не меньше 0.75 и время обучения и предсказания намного выше чем у модели LogisticRegression
* Модель LogisticRegression можно рекомендовать для использования в качестве инструмента, который будет искать токсичные комментарии и отправлять их на модерацию.

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'>Сергей, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Не применяй метод .astype('U') для текстов на латинице.</font></ul>
<ul><font color='red'>Поправь Лемматизацию.</font></ul>
<ul><font color='red'>Если используешь предикт для сравнения моделей и подбора гиперпараметров отдели для этого валидационную выборку или используй кроссвалидацию.</font></ul>

<font color='blue'>Что можно сделать лучше:</font>
<font color='orange'>В работе я оставил несколько советов. Буду рад, если ты учтешь их.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>

<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'>Сергей, большое спасибо за доработку.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Сделай загрузку моделей nltk один раз.</font></ul>

<font color='blue'><b>Присылай новую версию проекта через Пактикум, что бы можно было её зачесть.</b></font>

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны